# Setting up Calliope Technology Data

This notebook sets up the technology data for a Calliope model.

The data will be retrieved from NREL's Annual Technology Baseline using ``nrelpy``. The required data will be retrieved from Calliope's [documentation](https://calliope.readthedocs.io/en/stable/user/config_defaults.html#).

In [1]:
from nrelpy.atb import ATBe

In [2]:
import nrelpy
import pandas as pd
import numpy as np
import yaml
import matplotlib.pyplot as plt

In [3]:
frames = pd.read_html("https://calliope.readthedocs.io/en/stable/user/config_defaults.html#")

In [4]:
for f in frames:
    display(f.head(2))

,Setting,Default,Comments
0,calliope_version,NaN,Calliope framework version this model is inten...
1,group_share,{},Optional settings for the group_share constrai...


,Setting,Default,Comments
0,backend,pyomo,Backend to use to build and solve the model. A...
1,bigM,1000000000.0,"Used for unmet demand, but should be of a simi..."


,Setting,Default,Name,Unit,Comments
0,carrier_ratios,NaN,Carrier ratios,fraction,Ratio of summed output of carriers in [‘out_2’...
1,charge_rate,NaN,Charge rate,hour -1,"(do not use, replaced by energy_cap_per_storag..."


,Setting,Default,Name,Unit,Comments
0,energy_cap,0,Cost of energy capacity,kW gross -1,NaN
1,energy_cap_per_distance,0,"Cost of energy capacity, per unit distance",kW gross -1 / distance,Applied to transmission links only


In [5]:
operational_constraints = frames[2]
cost_constraints = frames[3]

In [6]:
atb = ATBe(year=2023)

In [7]:
techs = ["Coal",
"Hydro",
"Natural gas",
"Nuclear",
"Other",
"Solar",
"Wind",]

In [8]:
atb.index_names

['core_metric_case',
 'crpyears',
 'maturity',
 'scale',
 'scenario',
 'technology',
 'core_metric_parameter',
 'core_metric_variable']

In [9]:
nrel_techs = ['Coal_FE','Hydropower','NaturalGas_FE','Nuclear','UtilityPV','LandbasedWind']

In [10]:
coal_data = atb(scenario='Moderate',
    maturity='Y', 
    scale='Utility', 
    core_metric_case='Market', 
    crpyears=20, 
    core_metric_variable=2025,
    technology="Coal_FE")['Coal-new']
# coal_data

In [40]:
coal_tech = {"essentials":{'name':'Coal',
                           'parent':'supply',
                           'carrier':'power',
                           'color':'#000000'},
             'constraints':{'energy_prod':'true',
                            'energy_ramping':0.4,
                            'lifetime':75,
                            },
             'costs':{'monetary':{'interest_rate':0.10,
                                  'energy_cap':float(coal_data.loc['CAPEX']),
                                  'om_annual':float(coal_data.loc['Fixed O&M']),
                                  'om_prod':float(coal_data.loc['Variable O&M']+9.34)/1e3}}}

In [12]:
pv_data = atb(scenario='Moderate',
    maturity='Y', 
    scale='Utility', 
    core_metric_case='Market', 
    crpyears=20, 
    core_metric_variable=2025,
    technology="UtilityPV")['Utility PV - Class 7']
# pv_data

In [51]:
pv_tech = {"essentials":{'name':'UtilityPV',
                           'parent':'supply_plus',
                           'carrier':'power',
                           'color':'#fcd305'},
             'constraints':{'energy_prod':'true',
                            'lifetime':20,
                            'resource':'file=2023-miso-solar.csv',
                            'resource_area_max':16781678900000000.0,
                            'resource_unit':'energy',
                            # 'force_resource':'true'
                            },
             'costs':{'monetary':{'interest_rate':0.10,
                                  'energy_cap':float(pv_data.loc['CAPEX']),
                                  'om_annual':float(pv_data.loc['Fixed O&M'])}}}

In [14]:
wind_data = atb(scenario='Moderate',
    maturity='Y', 
    scale='Utility', 
    core_metric_case='Market', 
    crpyears=20, 
    core_metric_variable=2025,
    technology="LandbasedWind")['Land-Based Wind - Class 6 - Technology 1']
# wind_data

In [52]:
wind_tech = {"essentials":{'name':'LandbasedWind',
                           'parent':'supply_plus',
                           'carrier':'power',
                           'color':'#05fcf8'},
             'constraints':{'energy_prod':'true',
                            'lifetime':20,
                            'resource':'file=2023-miso-wind.csv',
                            'resource_area_max':16781678900000000.0,
                            'resource_unit':'energy',
                            # 'force_resource':'true'
                            },
             'costs':{'monetary':{'interest_rate':0.10,
                                  'energy_cap':float(wind_data.loc['CAPEX']),
                                  'om_annual':float(wind_data.loc['Fixed O&M'])}}}

In [16]:
natgas_data = atb(scenario='Moderate',
    maturity='Y', 
    scale='Utility', 
    core_metric_case='Market', 
    crpyears=20, 
    core_metric_variable=2025,
    technology="NaturalGas_FE")['NG Combined Cycle (F-Frame)']
# natgas_data

In [37]:
natgas_tech = {"essentials":{'name':'NaturalGas-CC',
                           'parent':'supply',
                           'carrier':'power',
                           'color':'#c77228'},
             'constraints':{'energy_prod':'true',
                            'energy_ramping':0.8,
                            'lifetime':50,
                            },
             'costs':{'monetary':{'interest_rate':0.10,
                                  'energy_cap':float(natgas_data.loc['CAPEX']),
                                  'om_annual':float(natgas_data.loc['Fixed O&M']),
                                  'om_prod':float(natgas_data.loc['Variable O&M']+12.45)/1e3}}}

In [18]:
hydro_data = atb(scenario='Moderate',
    maturity='Y', 
    scale='Utility', 
    core_metric_case='Market', 
    crpyears=20, 
    core_metric_variable=2025,
    technology="Hydropower")['Hydropower - NSD 3']
# hydro_data

In [36]:
hydro_tech = {"essentials":{'name':'Hydropower',
                           'parent':'supply_plus',
                           'carrier':'power',
                           'color':'#282dc8'},
             'constraints':{'energy_prod':'true',
                            'lifetime':100,
                            'resource_unit':'energy',
                            'resource_eff':float(hydro_data.loc['CF'])
                            },
             'costs':{'monetary':{'interest_rate':0.10,
                                  'energy_cap':float(hydro_data.loc['CAPEX']),
                                  'om_annual':float(hydro_data.loc['Fixed O&M'])}}}

In [20]:
nuclear_data = atb(scenario='Moderate',
    maturity='Y', 
    scale='Utility', 
    core_metric_case='Market', 
    crpyears=20, 
    core_metric_variable=2025,
    technology="Nuclear")['Nuclear - AP1000']
nuclear_data

core_metric_parameter
CAPEX           8106.020999
CFC             1693.702992
Fixed O&M        152.120000
Heat Rate         10.447000
OCC             6412.318007
Variable O&M       2.470000
CF                 0.927000
FCR                0.061003
LCOE              88.971842
Fuel               6.874484
Name: Nuclear - AP1000, dtype: float64

In [35]:
nuclear_tech = {"essentials":{'name':'Nuclear AP1000',
                           'parent':'supply',
                           'carrier':'power',
                           'color':'#097805'},
             'constraints':{'energy_prod':'true',
                            'energy_ramping':0.8,
                            'lifetime':60,
                            },
             'costs':{'monetary':{'interest_rate':0.10,
                                  'energy_cap':float(nuclear_data.loc['CAPEX']),
                                  'om_annual':float(nuclear_data.loc['Fixed O&M']),
                                  'om_prod':float(nuclear_data.loc['Variable O&M']+nuclear_data.loc['Fuel'])/1e3}}}

In [43]:
demand_tech = {"essentials":{"name":'Power demand',
                             "color":"#072486",
                             "parent":"demand",
                             "carrier":"power"}}

In [53]:
tech_data = {'ccgt':natgas_tech,
             'nuclear':nuclear_tech,
             'coal':coal_tech,
             'util-pv':pv_tech,
             'wind':wind_tech,
             'hydro':hydro_tech,
             'demand_power':demand_tech}
tech_file = {'techs':tech_data}

In [54]:
with open('model_config/miso_techs.yaml', 'w') as file:
    yaml.dump(tech_file, file)